# 財務データの作成

In [140]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

## (1) 企業データを年で縦に結合する

### 各年を接合する

In [141]:
# 各年の結合
common = '../../data/SPEEDA/CompanyList_1018/CompanyList_'
cols = [
    'stock_code', 'corporate_number', 'firm_name', 'fiscal_year', 'status', 'establishment', 'prefecture',
    'ceo', 'accounting', 'sales', 'operating_profit', 'net_profit', 'total_assets', 'cash',
    'fixed_assets', 'total_liabilities', 'current_liabilities', 'stockholders_equity', 'other_gains',
    'operating_cash_flow', 'stock_price', 'extraordinary_loss', 'foreign_stock_ratio'
    ]
df_list = []
for year in range(1998, 2021):
    path = common + str(year) + '.xlsx'
    df = pd.read_excel(path, sheet_name='企業リスト', skiprows=7, header=0)
    df.dropna(subset=['企業名称'], how='any', inplace=True) # 空行等を削除する
    df.set_axis(labels=cols, axis=1, inplace=True)
    df_list.append(df)
    # print(f'{year} was successfully closed')

df = pd.concat(df_list, axis=0) # 縦に結合
df.dropna(subset=['fiscal_year'], inplace=True) # 不要行の削除
df = df.assign( # 列の更新・修正（新規追加含む）
    year = df['fiscal_year'].apply(lambda x: str(x)[0:4]),
    month = df['fiscal_year'].apply(lambda x: str(x)[5:7]),
    establishment = df['establishment'].apply(lambda x: str(x)[0:4] if not x == '-' else np.nan),
    ceo = df['ceo'].apply(lambda x: x if not x == '-' else np.nan),
    prefecture = df['prefecture'].apply(lambda x: x if not x == '-' else np.nan),
    accounting_code = df['accounting'].astype('category').cat.codes.replace(-1, np.nan), # 会計基準コードの付与
    fixed_assets = df['fixed_assets'].fillna(0),
    stockholders_equity = df['stockholders_equity'].fillna(0),
    other_gains = df['other_gains'].fillna(0),
    operating_cash_flow = df['operating_cash_flow'].fillna(0),
    extraordinary_loss = df['extraordinary_loss'].fillna(0),
)

# 経営者データの更新
path = '../../data/eol/ceo_data.csv'
ceo_data = pd.read_csv(path, encoding='cp932', header=0)
ceo_data = ceo_data[['証券コード', 'データ年', '代表者名']]
ceo_data.set_axis(['stock_code', 'year', 'ceo_new'], inplace=True, axis=1)
ceo_data = ceo_data.astype({
    'stock_code': 'str',
    'year': 'str',
    'ceo_new': 'str'
})
df = pd.merge(left=df, right=ceo_data, on=['stock_code', 'year'], how='left')
df['ceo'] = df['ceo_new']

# 業種の付与
path = '../../data/TDnet/証券コードリスト/syoken_code.xls'
ind_data = pd.read_excel(path, sheet_name='Sheet1', header=0)
ind_data.columns = [
    'ymd', 'stock_code', 'firm_name', 'market', 'indcode_small', 'indname_small', 'indcode_large',
    'indname_large', 'scale_code', 'scale_category'
    ]
ind_data = ind_data[['stock_code', 'indcode_small', 'indname_small', 'indcode_large', 'indname_large']]
ind_data['stock_code'] = ind_data['stock_code'].apply(str)
for col in ['indcode_small', 'indname_small', 'indcode_large', 'indname_large']:
    ind_data[col] = ind_data[col].apply(lambda x: x if not x == '-' else np.nan)
df = pd.merge(left=df, right=ind_data, on='stock_code', how='left')

# 都道府県コードの付与
path = '../../data/Prefecture/prefecture_code.csv'
prefecture_code = pd.read_csv(path, encoding='cp932', header=0)
df = pd.merge(left=df, right=prefecture_code, on='prefecture', how='left')

# 感応度分析有無の付与
path = '../../data/eol/sensitivity/sensitivity_all.csv'
sensitivity = pd.read_csv(path, encoding='utf_8_sig', header=0)
sensitivity = sensitivity[['stock_code', 'year', 'sensitivity']]
sensitivity['stock_code'] = sensitivity['stock_code'].apply(str)
df = pd.merge(left=df, right=sensitivity, on=['stock_code', 'year'], how='left')
df['sensitivity'] = df['sensitivity'].fillna(0)

# BCPの付与
path = 'C:/Users/koeci/Google ドライブ/MBA/ワークショップ/data/eol/BCP/securities/BCP_data.csv'
bcp = pd.read_csv(path, encoding='cp932', header=0)
bcp = bcp[['stock_code', 'year', 'BCP', 'BCP_first_year']] # 列を選択
bcp['stock_code'] = bcp['stock_code'].apply(str) # 型変換
bcp['year'] = bcp['year'].apply(str) # 型変換
df = pd.merge(left=df, right=bcp, on=['stock_code', 'year'], how='left')

## 初めてBCPを策定した年以降にBCPを策定していることを表す１を付与する
df_grouped = df.groupby('stock_code')['BCP_first_year'].min()
df_grouped.name = 'fy'
df = pd.merge(left=df, right=df_grouped, on='stock_code', how='left')
df['BCP_first'] = [1 if int(y) == int(y_bcp) else 0 for y, y_bcp in zip(df['year'], df['fy'].fillna(0))]
df['BCP_dev'] = [1 if int(y) >= int(y_bcp) else 0 for y, y_bcp in zip(df['year'], df['fy'].fillna(10000))] # nanを一時的に補完（大きい数なら何でも良い）
df.drop(columns=['BCP_first_year', 'fy'], inplace=True)

## nanを0で埋める
for col in ['BCP', 'BCP_dev']:
    df[col] = df[col].fillna(0)

# 列名でソート
cols_sort = [
    'stock_code', 'firm_name', 'year', 'month', 'establishment', 'ceo', 'accounting', 'accounting_code', 'prefecture', 'prefecture_code',
    'indcode_small', 'indname_small', 'indcode_large', 'indname_large', 'sales', 'operating_profit',
    'net_profit', 'total_assets', 'cash', 'fixed_assets', 'total_liabilities', 'current_liabilities',
    'stockholders_equity', 'other_gains', 'operating_cash_flow', 'stock_price', 'extraordinary_loss', 'foreign_stock_ratio', 'sensitivity',
    'BCP', 'BCP_first', 'BCP_dev'
]
df = df.loc[:, cols_sort]
df.sort_values(['stock_code', 'year'], inplace=True)

# 保存(unicode error回避のためignoreを設定する)
path = '../../data/SPEEDA/CompanyList_1018/CompanyListBinded.csv'
with open(path, mode='w', encoding='cp932', errors='ignore') as file:
    df.to_csv(file, header=True, index=False)

### 1期前のデータを横にくっつける

In [142]:
path = '../../data/SPEEDA/CompanyList_1018/CompanyListBinded.csv'
df_pre = pd.read_csv(path, encoding='cp932', header=0)
df_grouped = df_pre.groupby('stock_code')
cols_pre = [
    'year', 'month', 'establishment', 'ceo', 'accounting', 'accounting_code', 'prefecture', 'prefecture_code',
    'sales', 'operating_profit', 'net_profit', 'total_assets', 'cash', 'fixed_assets', 'total_liabilities',
    'current_liabilities', 'stockholders_equity', 'other_gains', 'operating_cash_flow', 'stock_price',
    'extraordinary_loss', 'foreign_stock_ratio', 'sensitivity', 'BCP', 'BCP_first', 'BCP_dev'
]
for col in cols_pre:
    new_col = 'lag_' + col
    df_pre[new_col] = df_grouped[col].shift(1)

path = '../../data/SPEEDA/CompanyList_1018/CompanyListBinded_withPre.csv'
df_pre.to_csv(path, encoding='cp932', header=True, index=False)

## (2) 経営指標を作成する

### 指標の作成

In [143]:
# データの読み込み
path = '../../data/SPEEDA/CompanyList_1018/CompanyListBinded_withPre.csv'
df = pd.read_csv(path, header=0, encoding='cp932')

# 指標作成に当たり、割り算の分母が０になるのを未然に防ぐ
cols = [
    'sales', 'operating_profit', 'operating_cash_flow', 'total_assets', 'stockholders_equity',
    'other_gains', 'lag_stock_price', 'lag_net_profit'
    ]
for col in cols:
    df[col] = df[col].apply(lambda x: x if not x == 0 else np.nan)

# 指標の作成
df_b = df.assign(
    b_sales_growth = (df['sales'] - df['lag_sales']) / df['lag_sales'],
    b_total_assets = df['total_assets'],
    b_operating_cash_flow = df['operating_cash_flow'],
    b_ros = df['operating_profit'] / df['sales'],
    b_cash_deposit_ratio = df['cash'] / (df['stockholders_equity'] + df['other_gains']),
    b_leverage = df['total_assets'] / (df['stockholders_equity'] + df['other_gains']),
    b_stock_price_growth = (df['stock_price'] - df['lag_stock_price']) / df['lag_stock_price'],
    b_net_profit_growth = (df['net_profit'] - df['lag_net_profit']) / df['lag_net_profit'],
    b_firm_age = df['year'] - df['establishment'],
    b_fixed_assets_ratio = df['fixed_assets'] / (df['stockholders_equity'] + df['other_gains']),
    b_extraordinary_loss = df['extraordinary_loss'],
    b_foreign_stock_ratio = df['foreign_stock_ratio'],
    b_earthquake = [1 if (y >= 2012) | ((y == 2011) & (m >= 4)) else 0 for y, m in zip(df['year'], df['month'])],
    b_covid_19 = [1 if ((y == 2020) & (m >= 2)) | (y >= 2021) else 0 for y, m in zip(df['year'], df['month'])],
    b_turnover = [1 if not ceo == lag_ceo else 0 for ceo, lag_ceo in zip(df['ceo'], df['lag_ceo'])],
    b_sensitivity_analysis = df['sensitivity']
)

# 経営者交代の計測に前期分を使うので、前期がない行を落とす
df_b.dropna(subset=['lag_ceo'], inplace=True)

# 計算できなかった指標をNAにする
df_b['b_firm_age'] = df_b['b_firm_age'].apply(lambda x: x if x >= 0 else np.nan)
# df_b['b_net_profit_ratio'] = df_b['b_net_profit_ratio'].apply(lambda x: x if not np.isinf(x) else np.nan)

### NAなどの処理&グループ化計算が必要な指標の計算

In [144]:
# NaNは全て落とす
df_b.dropna(how='any', inplace=True)

# 異常値処理
def remove_outliers(df, col, q_min=0, q_max=1):
    lower = df[col].quantile(q_min)
    upper = df[col].quantile(q_max)
    df = df.query(f'{lower} <= {col} & {col} <= {upper}')

    return df

cols = ['b_operating_cash_flow', 'b_fixed_assets_ratio']
for col in cols:
    df_b = remove_outliers(df_b, col, q_min=0.005, q_max=0.995)
    
# 経営者累積交代回数の計測
df_b['b_turnover_cumsum'] = df_b.groupby('stock_code').cumsum()['b_turnover']

# 感応度分析累積実施回数の計測
df_b['b_sensitivity_analysis_cumsum'] = df_b.groupby('stock_code').cumsum()['b_sensitivity_analysis']

# BCP累計開示回数の計測
df_b['b_BCP_cumsum'] = df_b.groupby('stock_code').cumsum()['BCP']

# 1期前の指標を横にくっつける
cols = [c for c in df_b.columns if c.startswith('b_')]
df_grouped = df_b.groupby('stock_code')
for col in cols:
    new_col = 'lag_' + col
    df_b[new_col] = df_grouped[col].shift(1)

df_b.dropna(how='any', inplace=True)

# 年数が足りない企業＆年が飛んでいる企業を落とす
df_grouped = df_b.groupby('stock_code').agg(['count', 'min']) # 企業ごとのレコード数をカウント
df_grouped = df_grouped['year'] # year列を抽出（これをしないとエラー）
df_b = pd.merge(left=df_b, right=df_grouped, how='left', on='stock_code') # 元データにカウント数をマージする
df_b['row'] = df_b.groupby('stock_code').cumcount() # グループごとに連番を振る
df_b['correct_year'] = df_b['min'] + df_b['row'] # 本来あるべき年
df_b = df_b.query('year == correct_year') # 年が飛んでいる企業を落とす

print(df_b['stock_code'].drop_duplicates().count()) # 企業数（ユニーク）の表

# 保存
path = '../../data/SPEEDA/CompanyList_1018/CompanyListProcessed.csv'
df_b.to_csv(path, header=True, index=False, encoding='cp932')

3124


### Yeo-Johnson変換

In [145]:
def trans_yeo_johnson(series):
    mm = MinMaxScaler()
    pt = PowerTransformer(standardize=True)
    data = series.values.reshape(-1, 1)
    mm.fit(data)
    pt.fit(data)
    result = pt.transform(data)

    return result

cols = [c for c in df_b.columns if ('b_' in c)]
for col in cols:
    new_col = col.replace('b_', 'yj_')
    df_b[new_col] = trans_yeo_johnson(df_b[col])
    
df_b.rename(columns={
    'b_earthquake': 'earthquake', 'lag_b_earthquake': 'lag_earthquake',
    'b_turnover': 'turnover', 'lag_b_turnover': 'lag_turnover',
    'b_turnover_cumsum': 'turnover_cumsum', 'lag_b_turnover_cumsum': 'lag_turnover_cumsum',
    'b_sensitivity_analysis': 'sensitivity_analysis', 'lag_b_sensitivity_analysis': 'lag_sensitivity_analysis',
    'b_sensitivity_analysis_cumsum': 'sensitivity_analysis_cumsum', 'lag_b_sensitivity_analysis_cumsum': 'lag_sensitivity_analysis_cumsum',
    'b_covid_19': 'covid_19', 'lag_b_covid_19': 'lag_covid_19',
    'b_BCP_cumsum': 'BCP_cumsum', 'lag_b_BCP_cumsum': 'lag_BCP_cumsum'
    }, inplace=True)
cols = [
    'stock_code', 'firm_name', 'year', 'month', 'establishment', 'prefecture', 'prefecture_code',
    'ceo', 'accounting', 'accounting_code',
    'indcode_small', 'indname_small', 'indcode_large', 'indname_large'
    ]
cols = cols \
    + [c for c in df_b.columns if ('yj_' in c) & ~('earthquake' in c) & ~('turnover' in c) & ~('sensitivity' in c) & ~('covid_19' in c) & ~('BCP' in c)] \
    + ['earthquake', 'lag_earthquake', 'turnover', 'lag_turnover', 'turnover_cumsum', 'lag_turnover_cumsum', 'sensitivity_analysis', 'lag_sensitivity_analysis', \
    'sensitivity_analysis_cumsum', 'lag_sensitivity_analysis_cumsum', 'covid_19', 'lag_covid_19', 
    'BCP', 'lag_BCP', 'BCP_cumsum', 'lag_BCP_cumsum', 'BCP_dev', 'lag_BCP_dev', 'b_sales_growth']
df_yj = df_b[cols]
print(df_yj.columns)
path = '../../data/Analysis/analysis_data.csv'
df_yj.to_csv(path, header=True, index=False, encoding='cp932')

Index(['stock_code', 'firm_name', 'year', 'month', 'establishment',
       'prefecture', 'prefecture_code', 'ceo', 'accounting', 'accounting_code',
       'indcode_small', 'indname_small', 'indcode_large', 'indname_large',
       'yj_sales_growth', 'yj_total_assets', 'yj_operating_cash_flow',
       'yj_ros', 'yj_cash_deposit_ratio', 'yj_leverage',
       'yj_stock_price_growth', 'yj_net_profit_growth', 'yj_firm_age',
       'yj_fixed_assets_ratio', 'yj_extraordinary_loss',
       'yj_foreign_stock_ratio', 'lag_yj_sales_growth', 'lag_yj_total_assets',
       'lag_yj_operating_cash_flow', 'lag_yj_ros', 'lag_yj_cash_deposit_ratio',
       'lag_yj_leverage', 'lag_yj_stock_price_growth',
       'lag_yj_net_profit_growth', 'lag_yj_firm_age',
       'lag_yj_fixed_assets_ratio', 'lag_yj_extraordinary_loss',
       'lag_yj_foreign_stock_ratio', 'earthquake', 'lag_earthquake',
       'turnover', 'lag_turnover', 'turnover_cumsum', 'lag_turnover_cumsum',
       'sensitivity_analysis', 'lag_sensit

## 決算期を３月に限定したver

In [146]:
path = '../../data/Analysis/analysis_data.csv'
df = pd.read_csv(path, encoding='cp932', header=0)
df = df.query('month == 3')
df.dropna(how='any', inplace=True) # 影響なし
path = '../../data/Analysis/analysis_data_three.csv'
df.to_csv(path, encoding='cp932', header=True, index=False)

## サンプル数チェック

In [147]:
path = '../../data/Analysis/analysis_data.csv'
df = pd.read_csv(path, encoding='cp932', header=0)
path = '../../data/Analysis/analysis_data_three.csv'
df_three = pd.read_csv(path, encoding='cp932', header=0)

def sample_check(df):
    firmnum = df.drop_duplicates(subset=['stock_code'])['stock_code'].count()
    bcp_firmnum = df.query('BCP == 1').drop_duplicates(subset=['stock_code'])['stock_code'].count()
    sensitivity_firmnum = df.query('sensitivity_analysis == 1').drop_duplicates(subset=['stock_code'])['stock_code'].count()
    turnover_firmnum = df.query('turnover == 1').drop_duplicates(subset=['stock_code'])['stock_code'].count()
    print(f'企業数：{firmnum}')
    print(f'BCP策定企業数：{bcp_firmnum}')
    print(f'感応度分析実施企業数：{sensitivity_firmnum}')
    print(f'経営者交代企業数：{turnover_firmnum}')

df_list = [df, df_three]
annot = ['全期間', '3月']
for d, a in zip(df_list, annot):
    print(f'##### {a} #####')
    sample_check(d)

##### 全期間 #####
企業数：3124
BCP策定企業数：1174
感応度分析実施企業数：360
経営者交代企業数：1985
##### 3月 #####
企業数：2119
BCP策定企業数：926
感応度分析実施企業数：308
経営者交代企業数：1507


## データの作成メモ
- outlier: 分布が極端に偏る変数について、上下0.5%ずつの異常値を除去
- year: 年数が飛んでいたり、５年以上の年数が確保できない企業を除去
- standarlized：多くの変数で分布が極端に偏っているため、Yeo-Johnson変換を実施。Box-Cox変換と比べて、負の値を処理できる点から採用した。